#Version 1 (with Download - not working properly)

In [ ]:
# Install required libraries
!apt-get install -y tesseract-ocr
!pip install pytesseract easyocr opencv-python matplotlib gradio reportlab

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
import cv2
import pytesseract
import easyocr
import numpy as np
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import csv
import io
import os


In [ ]:
def ocr_with_tesseract(image):
    # Convert image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Perform OCR using Tesseract
    boxes = pytesseract.image_to_data(image_rgb)
    identified_words_tesseract = []
    confidence_scores_tesseract = []

    # Draw rectangles around detected words and collect identified words
    for i, box in enumerate(boxes.splitlines()):
        if i == 0:
            continue  # Skip the header
        b = box.split()
        if len(b) == 12:  # Ensure there are enough elements
            text = b[11]  # Extract the detected text
            conf = int(b[10])  # Get the confidence score
            if text.strip():  # Check if text is not empty
                x, y, width, height = int(b[6]), int(b[7]), int(b[8]), int(b[9])
                cv2.rectangle(image_rgb, (x, y), (x + width, y + height), (0, 255, 0), 2)
                identified_words_tesseract.append(text)
                confidence_scores_tesseract.append(conf)

    # Create output image
    output_image_tesseract = image_rgb
    return output_image_tesseract, identified_words_tesseract, confidence_scores_tesseract


In [ ]:
def ocr_with_easyocr(image):
    # Convert image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Initialize EasyOCR Reader
    reader = easyocr.Reader(['en'])  # Specify the language (English)

    # Perform OCR
    results = reader.readtext(image_rgb)
    identified_words_easyocr = []
    confidence_scores_easyocr = []

    # Draw rectangles around detected words and collect identified words
    for (bbox, text, prob) in results:
        (top_left, top_right, bottom_right, bottom_left) = bbox
        top_left = tuple(map(int, top_left))
        bottom_right = tuple(map(int, bottom_right))

        # Draw rectangle around detected text
        cv2.rectangle(image_rgb, top_left, bottom_right, (0, 255, 0), 2)

        # Append identified words and confidence
        identified_words_easyocr.append(text)
        confidence_scores_easyocr.append(prob)

    # Create output image
    output_image_easyocr = image_rgb
    return output_image_easyocr, identified_words_easyocr, confidence_scores_easyocr


In [ ]:
def compare_ocr(image):
    # Run both OCR functions
    output_image_tesseract, words_tesseract, scores_tesseract = ocr_with_tesseract(image)
    output_image_easyocr, words_easyocr, scores_easyocr = ocr_with_easyocr(image)

    # Create tabular output for Tesseract
    tesseract_table = gr.Dataframe([words_tesseract, scores_tesseract], headers=['Tesseract Words', 'Tesseract Confidence Scores'])

    # Create tabular output for EasyOCR
    easyocr_table = gr.Dataframe([words_easyocr, scores_easyocr], headers=['EasyOCR Words', 'EasyOCR Confidence Scores'])

    # Create download button for Tesseract results
    def download_tesseract_results():
        # Create a CSV file in memory
        csv_buffer = io.StringIO()
        writer = csv.writer(csv_buffer)
        writer.writerow(['Tesseract Words', 'Tesseract Confidence Scores'])
        writer.writerows(zip(words_tesseract, scores_tesseract))
        return csv_buffer.getvalue(), "tesseract_results.csv"


    tesseract_download = gr.File(label="Download Tesseract Results", file_types=["csv"], interactive=download_tesseract_results)

    # Create download button for EasyOCR results
    def download_easyocr_results():
        # Create a CSV file in memory
        csv_buffer = io.StringIO()
        writer = csv.writer(csv_buffer)
        writer.writerow(['EasyOCR Words', 'EasyOCR Confidence Scores'])
        writer.writerows(zip(words_easyocr, scores_easyocr))
        return csv_buffer.getvalue(), "easyocr_results.csv"

    easyocr_download = gr.File(label="Download EasyOCR Results", file_types=["csv"], interactive=download_easyocr_results)

    return [
        gr.Image(output_image_tesseract, label="Tesseract Output"),
        tesseract_table,
        tesseract_download,
        gr.Image(output_image_easyocr, label="EasyOCR Output"),
        easyocr_table,
        easyocr_download
    ]

iface = gr.Interface(
    fn=compare_ocr,
    inputs=gr.Image(type="numpy"),
    outputs=[
        gr.Image(type="numpy"),
        gr.Dataframe(),
        gr.Button("Download Tesseract Results"),
        gr.Image(type="numpy"),
        gr.Dataframe(),
        gr.Button("Download easyOCR Results")
    ],
    title="OCR Comparison: Tesseract vs EasyOCR",
    description="Upload an image to perform OCR using both Tesseract and EasyOCR. Compare the results side by side."
)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ba003072fd25f0254c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#Version 2 (without Download - final and working proper)

In [ ]:
# Install required libraries
!apt-get install -y tesseract-ocr
!pip install pytesseract easyocr opencv-python matplotlib gradio reportlab


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (3,799 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123632 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [ ]:
import cv2
import pytesseract
import easyocr
import numpy as np
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import csv
import io
import os

In [ ]:
def ocr_with_tesseract(image):
    # Convert image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Perform OCR using Tesseract
    boxes = pytesseract.image_to_data(image_rgb)
    identified_words_tesseract = []
    confidence_scores_tesseract = []

    # Draw rectangles around detected words and collect identified words
    for i, box in enumerate(boxes.splitlines()):
        if i == 0:
            continue  # Skip the header
        b = box.split()
        if len(b) == 12:  # Ensure there are enough elements
            text = b[11]  # Extract the detected text
            conf = int(b[10])  # Get the confidence score
            if text.strip():  # Check if text is not empty
                x, y, width, height = int(b[6]), int(b[7]), int(b[8]), int(b[9])
                cv2.rectangle(image_rgb, (x, y), (x + width, y + height), (0, 255, 0), 2)
                identified_words_tesseract.append(text)
                confidence_scores_tesseract.append(conf)

    # Create output image
    output_image_tesseract = image_rgb
    return output_image_tesseract, identified_words_tesseract, confidence_scores_tesseract

def ocr_with_easyocr(image):
    # Convert image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Initialize EasyOCR Reader
    reader = easyocr.Reader(['en'])  # Specify the language (English)

    # Perform OCR
    results = reader.readtext(image_rgb)
    identified_words_easyocr = []
    confidence_scores_easyocr = []

    # Draw rectangles around detected words and collect identified words
    for (bbox, text, prob) in results:
        (top_left, top_right, bottom_right, bottom_left) = bbox
        top_left = tuple(map(int, top_left))
        bottom_right = tuple(map(int, bottom_right))

        # Draw rectangle around detected text
        cv2.rectangle(image_rgb, top_left, bottom_right, (0, 255, 0), 2)

        # Append identified words and confidence
        identified_words_easyocr.append(text)
        confidence_scores_easyocr.append(prob)

    # Create output image
    output_image_easyocr = image_rgb
    return output_image_easyocr, identified_words_easyocr, confidence_scores_easyocr

def compare_ocr(image):
    # Run both OCR functions
    output_image_tesseract, words_tesseract, scores_tesseract = ocr_with_tesseract(image)
    output_image_easyocr, words_easyocr, scores_easyocr = ocr_with_easyocr(image)

    # Create tabular output
    tesseract_table = gr.Dataframe(
        pd.DataFrame({'Words': words_tesseract, 'Confidence Scores': scores_tesseract})
    )
    easyocr_table = gr.Dataframe(
        pd.DataFrame({'Words': words_easyocr, 'Confidence Scores': scores_easyocr})
    )

    # Create download buttons
    def download_tesseract_results():
        # Create a CSV file in memory
        csv_buffer = io.StringIO()
        writer = csv.writer(csv_buffer)
        writer.writerow(['Words', 'Confidence Scores'])
        writer.writerows(zip(words_tesseract, scores_tesseract))
        return csv_buffer.getvalue(), "tesseract_results.csv"

    def download_easyocr_results():
        # Create a CSV file in memory
        csv_buffer = io.StringIO()
        writer = csv.writer(csv_buffer)
        writer.writerow(['Words', 'Confidence Scores'])
        writer.writerows(zip(words_easyocr, scores_easyocr))
        return csv_buffer.getvalue(), "easyocr_results.csv"

    tesseract_download = gr.File(label="Download Tesseract Results", file_types=["csv"], interactive=download_tesseract_results)
    easyocr_download = gr.File(label="Download EasyOCR Results", file_types=["csv"], interactive=download_easyocr_results)

    return [
        gr.Image(output_image_tesseract, label="Tesseract Output"),
        tesseract_table,
        gr.Image(output_image_easyocr, label="EasyOCR Output"),
        easyocr_table
    ]

iface = gr.Interface(
    fn=compare_ocr,
    inputs=gr.Image(type="numpy"),
    outputs=[
        gr.Image(type="numpy"),
        gr.Dataframe(),
        gr.Image(type="numpy"),
        gr.Dataframe()
    ],
    title="OCR Comparison: Tesseract vs EasyOCR",
    description="Upload an image to perform OCR using both Tesseract and EasyOCR. Compare the results side by side."
)

iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://240d1bdced74b0c69c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#Version 3 (trail)

In [ ]:
# Install required libraries
!apt-get install -y tesseract-ocr
!pip install pytesseract easyocr opencv-python matplotlib gradio reportlab



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
import cv2
import pytesseract
import easyocr
import pandas as pd
import numpy as np
import gradio as gr
import matplotlib.pyplot as plt
import csv
import io

In [ ]:
def ocr_with_tesseract(image):
    # Convert image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Perform OCR using Tesseract
    boxes = pytesseract.image_to_data(image_rgb)
    identified_words_tesseract = []
    confidence_scores_tesseract = []

    # Draw rectangles around detected words and collect identified words
    for i, box in enumerate(boxes.splitlines()):
        if i == 0:
            continue  # Skip the header
        b = box.split()
        if len(b) == 12:  # Ensure there are enough elements
            text = b[11]  # Extract the detected text
            conf = int(b[10])  # Get the confidence score
            if text.strip():  # Check if text is not empty
                x, y, width, height = int(b[6]), int(b[7]), int(b[8]), int(b[9])
                cv2.rectangle(image_rgb, (x, y), (x + width, y + height), (0, 255, 0), 2)
                identified_words_tesseract.append(text)
                confidence_scores_tesseract.append(conf)

    # Create output image
    output_image_tesseract = image_rgb
    return output_image_tesseract, identified_words_tesseract, confidence_scores_tesseract

def ocr_with_easyocr(image):
    # Convert image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Initialize EasyOCR Reader
    reader = easyocr.Reader(['en'])  # Specify the language (English)

    # Perform OCR
    results = reader.readtext(image_rgb)
    identified_words_easyocr = []
    confidence_scores_easyocr = []

    # Draw rectangles around detected words and collect identified words
    for (bbox, text, prob) in results:
        (top_left, top_right, bottom_right, bottom_left) = bbox
        top_left = tuple(map(int, top_left))
        bottom_right = tuple(map(int, bottom_right))

        # Draw rectangle around detected text
        cv2.rectangle(image_rgb, top_left, bottom_right, (0, 255, 0), 2)

        # Append identified words and confidence
        identified_words_easyocr.append(text)
        confidence_scores_easyocr.append(prob)

    # Create output image
    output_image_easyocr = image_rgb
    return output_image_easyocr, identified_words_easyocr, confidence_scores_easyocr

def compare_ocr(image):
    # Run both OCR functions
    output_image_tesseract, words_tesseract, scores_tesseract = ocr_with_tesseract(image)
    output_image_easyocr, words_easyocr, scores_easyocr = ocr_with_easyocr(image)

    # Create DataFrames for Tesseract and EasyOCR results
    tesseract_df = pd.DataFrame({'Words': words_tesseract, 'Confidence Scores': scores_tesseract})
    easyocr_df = pd.DataFrame({'Words': words_easyocr, 'Confidence Scores': scores_easyocr})

    # Create CSV output in memory for Tesseract
    def download_tesseract_results():
        csv_buffer = io.StringIO()
        tesseract_df.to_csv(csv_buffer, index=False)
        return csv_buffer.getvalue(), "tesseract_results.csv"

    # Create CSV output in memory for EasyOCR
    def download_easyocr_results():
        csv_buffer = io.StringIO()
        easyocr_df.to_csv(csv_buffer, index=False)
        return csv_buffer.getvalue(), "easyocr_results.csv"

    # Create tables for display
    tesseract_table = gr.Dataframe(tesseract_df)
    easyocr_table = gr.Dataframe(easyocr_df)

    return [
        gr.Image(output_image_tesseract, label="Tesseract Output"),
        tesseract_table,
        gr.File(label="Download Tesseract Results", file_types=["csv"], interactive=download_tesseract_results),
        gr.Image(output_image_easyocr, label="EasyOCR Output"),
        easyocr_table,
        gr.File(label="Download EasyOCR Results", file_types=["csv"], interactive=download_easyocr_results)
    ]

iface = gr.Interface(
    fn=compare_ocr,
    inputs=gr.Image(type="numpy"),
    outputs=[
        gr.Image(type="numpy"),
        gr.Dataframe(),
        gr.File(label="Download Tesseract Results", file_types=["csv"]),
        gr.Image(type="numpy"),
        gr.Dataframe(),
        gr.File(label="Download EasyOCR Results", file_types=["csv"])
    ],
    title="OCR Comparison: Tesseract vs EasyOCR",
    description="Upload an image to perform OCR using both Tesseract and EasyOCR. Compare the results side by side."
)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0f285e245cef09a475.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
